In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/Media_Bias_Feature_for_the_Identification_of_Biased_Statements_in_News_Articles')
!pwd

/content
/content/drive/MyDrive/Colab Notebooks/Media_Bias_Feature_for_the_Identification_of_Biased_Statements_in_News_Articles


In [ ]:
!pip install -U scikit-learn
!pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.6/720.6 kB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 15.2 MB/s eta 0:00:00


In [ ]:
# import all of the packages that will be used

# basic packages for data processing
import pandas as pd
import numpy as np
import gc

# packege for calculation
import math

# packages for machine learning
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.naive_bayes import GaussianNB, ComplementNB

# packages to interpret the training result
import torch
from torchmetrics.classification import Accuracy, ConfusionMatrix, AUROC, Precision, Recall, F1Score

# package to benchmark the CPU running time
from time import process_time

# silent the warning from sklearn package
# import warnings
# warnings.filterwarnings('ignore')

# set the random seed to ensure the result is reproducible
import random
random.seed(42)
np.random.seed(42)

In [ ]:
fdt_hb_BABESG2_train = pd.read_csv('Ground_truth_datasets_characterised/Splits/Train/BABESG2_hybrid_characterised_index_output_train.csv')
fdt_hb_BABESG2_test = pd.read_csv('Ground_truth_datasets_characterised/Splits/Test/BABESG2_hybrid_characterised_index_output_test.csv')

In [ ]:
fdt_hb_BABESG2_train.columns

Index(['text', 'bias', 'bias_BABESG2', 'objective_BABESG2',
       'partly-subjective_BABESG2', 'subjective_BABESG2',
       'nonpartisan_headline', 'slight-partisan_headline',
       'hyperpartisan_headline', 'nonpartisan_article',
       'slight-partisan_article', 'hyperpartisan_article', 'true_LIAR',
       'mostly-true_LIAR', 'half-true_LIAR', 'barely-true_LIAR', 'false_LIAR',
       'pants-fire_LIAR', 'bluff_FakeNewsTFG', 'negative_NewsMTSC',
       'neutral_NewsMTSC', 'positive_NewsMTSC'],
      dtype='object')

In [ ]:
FEATURES = ['bias_BABESG2', 'objective_BABESG2',
       'partly-subjective_BABESG2', 'subjective_BABESG2',
       'nonpartisan_headline', 'slight-partisan_headline',
       'hyperpartisan_headline', 'nonpartisan_article',
       'slight-partisan_article', 'hyperpartisan_article', 'true_LIAR',
       'mostly-true_LIAR', 'half-true_LIAR', 'barely-true_LIAR', 'false_LIAR',
       'pants-fire_LIAR', 'bluff_FakeNewsTFG', 'negative_NewsMTSC',
       'neutral_NewsMTSC', 'positive_NewsMTSC']
TARGET = ['bias']

In [ ]:
train_X = fdt_hb_BABESG2_train[FEATURES]
test_X = fdt_hb_BABESG2_test[FEATURES]

train_y = fdt_hb_BABESG2_train['bias']
test_y = fdt_hb_BABESG2_test['bias']

In [ ]:
scaler = StandardScaler()
scaler.fit(train_X)

StandardScaler()

In [ ]:
train_X_scaled = scaler.transform(train_X)
test_X_scaled = scaler.transform(test_X)

In [ ]:
def runKFold(clf, output_type, f = 5, scaler = False):

  scaler = StandardScaler()

  val_acc = []
  val_prec = []
  val_rec = []
  val_f1 = []
  val_auroc = []

  for k in range(1, 6):
    fdt_hb_BABESG2_train_K = pd.read_csv(f'Ground_truth_datasets_characterised/Splits/Train/BABESG2_hybrid_characterised_{output_type}_output_train_KFold{k}.csv')
    fdt_hb_BABESG2_val_K = pd.read_csv(f'Ground_truth_datasets_characterised/Splits/Validation/BABESG2_hybrid_characterised_{output_type}_output_val_KFold{k}.csv')

    train_X = fdt_hb_BABESG2_train_K[FEATURES]
    val_X = fdt_hb_BABESG2_val_K[FEATURES]
    train_y = fdt_hb_BABESG2_train_K['bias']
    val_y = fdt_hb_BABESG2_val_K['bias']

    if(scaler == True):
      scaler.fit(train_X)
      train_X = scaler.transform(train_X)
      train_X = scaler.transform(val_X)

    trained_clf = clf.fit(train_X, train_y)

    preds = trained_clf.predict(val_X)
    preds = torch.from_numpy(np.array(preds))

    val_y = torch.from_numpy(val_y.to_numpy())

    accuracy = Accuracy(task = 'binary', threshold=0.5)
    precision = Precision(task = 'binary', threshold=0.5)
    recall = Recall(task = 'binary', threshold=0.5)
    f1 = F1Score(task = 'binary', threshold=0.5)
    auroc = AUROC(task = 'binary')

    val_acc.append(accuracy(preds, val_y).item())
    val_prec.append(precision(preds, val_y).item())
    val_rec.append(recall(preds, val_y).item())
    val_f1.append(f1(preds, val_y).item())
    val_auroc.append(auroc(preds, val_y).item())

    k += 1

    gc.collect()

  return val_acc, val_prec, val_rec, val_f1, val_auroc


# Gaussian Naive Bayes (GaussianNB)

**5 K-Fold Cross Validation (CV)**

In [ ]:
clf_GaussianNB = GaussianNB(var_smoothing = 0.004937047852839003)

In [ ]:
val_acc, val_prec, val_rec, val_f1, val_auroc = runKFold(clf_GaussianNB, 'index', 5, True)

In [ ]:
val_acc

[0.8174757361412048,
 0.801556408405304,
 0.7801556587219238,
 0.7918288111686707,
 0.7704280018806458]

In [ ]:
val_prec

[0.802281379699707,
 0.7937743067741394,
 0.7642585635185242,
 0.7799227833747864,
 0.7537878751754761]

In [ ]:
val_rec

[0.8339921236038208,
 0.8063241243362427,
 0.7976190447807312,
 0.8015872836112976,
 0.7896825671195984]

In [ ]:
val_f1

[0.817829430103302,
 0.800000011920929,
 0.7805825471878052,
 0.790606677532196,
 0.7713178396224976]

In [ ]:
val_auroc

[0.8177594542503357,
 0.8016294836997986,
 0.780488908290863,
 0.7920150756835938,
 0.7707955837249756]

In [ ]:
acc_cv_mean = np.mean(val_acc)
prec_cv_mean = np.mean(val_prec)
rec_cv_mean = np.mean(val_rec)
f1_cv_mean = np.mean(val_f1)
auroc_cv_mean = np.mean(val_auroc)

print('5-Fold CV Mean')
print('5-Fold CV Accuracy: {}'.format(acc_cv_mean))
print('5-Fold CV Precision: {}'.format(prec_cv_mean))
print('5-Fold CV Recall: {}'.format(rec_cv_mean))
print('5-Fold CV F1 Score: {}'.format(f1_cv_mean))
print('5-Fold CV AUROC: {}'.format(auroc_cv_mean))

5-Fold CV Mean
5-Fold CV Accuracy: 0.7922889232635498
5-Fold CV Precision: 0.7788049817085266
5-Fold CV Recall: 0.8058410286903381
5-Fold CV F1 Score: 0.7920673012733459
5-Fold CV AUROC: 0.7925377011299133


In [ ]:
acc_cv_std = np.std(val_acc)
prec_cv_std = np.std(val_prec)
rec_cv_std = np.std(val_rec)
f1_cv_std = np.std(val_f1)
auroc_cv_std = np.std(val_auroc)

print('5-Fold CV Standard Deviation')
print('5-Fold CV Accuracy: {}'.format(acc_cv_std))
print('5-Fold CV Precision: {}'.format(prec_cv_std))
print('5-Fold CV Recall: {}'.format(rec_cv_std))
print('5-Fold CV F1 Score: {}'.format(f1_cv_std))
print('5-Fold CV AUROC: {}'.format(auroc_cv_std))

5-Fold CV Standard Deviation
5-Fold CV Accuracy: 0.016404813434313038
5-Fold CV Precision: 0.017966220178823426
5-Fold CV Recall: 0.01509641529086539
5-Fold CV F1 Score: 0.01606977496412513
5-Fold CV AUROC: 0.016352146992069216


In [ ]:
cv_mean = ["cv mean", acc_cv_mean, prec_cv_mean, rec_cv_mean, f1_cv_mean, auroc_cv_mean]
cv_std = ["cv std", acc_cv_std, prec_cv_std, rec_cv_std, f1_cv_std, auroc_cv_std]

In [ ]:
five_k_cv_result_data = {'k': [1, 2, 3, 4, 5],
                          'val_acc': val_acc,
                          'val_prec': val_prec,
                          'val_rec': val_rec,
                          'val_f1': val_f1,
                          'val_auroc': val_auroc}

five_k_cv_result_df = pd.DataFrame(five_k_cv_result_data)

In [ ]:
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)

<ipython-input-47-f8ca0e644d46>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
<ipython-input-47-f8ca0e644d46>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)


In [ ]:
five_k_cv_result_df

,k,val_acc,val_prec,val_rec,val_f1,val_auroc
0,1,0.817476,0.802281,0.833992,0.817829,0.817759
1,2,0.801556,0.793774,0.806324,0.800000,0.801629
2,3,0.780156,0.764259,0.797619,0.780583,0.780489
3,4,0.791829,0.779923,0.801587,0.790607,0.792015
4,5,0.770428,0.753788,0.789683,0.771318,0.770796
5,cv mean,0.792289,0.778805,0.805841,0.792067,0.792538
6,cv std,0.016405,0.017966,0.015096,0.016070,0.016352


**Final Build**

In [ ]:
train_X = fdt_hb_BABESG2_train[FEATURES]
test_X = fdt_hb_BABESG2_test[FEATURES]

train_y = fdt_hb_BABESG2_train['bias']
test_y = fdt_hb_BABESG2_test['bias']

In [ ]:
scaler = StandardScaler()
scaler.fit(train_X)

StandardScaler()

In [ ]:
train_X_scaled = scaler.transform(train_X)
test_X_scaled = scaler.transform(test_X)

In [ ]:
train_X_scaled

array([[ 1.09857361,  0.82123887, -0.38594932, ...,  0.68041611,
        -0.4433099 , -0.45392215],
       [-0.94319561,  1.02758706, -0.63815385, ..., -1.64119106,
         2.71539819, -0.44902618],
       [-0.20775317,  1.02905762, -0.6395835 , ...,  0.6726993 ,
        -0.43187903, -0.45477963],
       ...,
       [-1.08940621,  1.02888928, -0.63963513, ...,  0.6157185 ,
        -0.35572074, -0.45337634],
       [-0.92479045, -1.05955536, -0.61476889, ..., -0.24535882,
        -0.44375651,  0.72981927],
       [ 1.20039837, -1.05913148, -0.57647161, ...,  0.68465581,
        -0.44946105, -0.45357213]])

In [ ]:
clf_GaussianNB = GaussianNB(var_smoothing = 0.004937047852839003)

In [ ]:
trained_clf = clf_GaussianNB.fit(train_X_scaled, train_y)

In [ ]:
preds = trained_clf.predict(test_X_scaled)

In [ ]:
preds = torch.from_numpy(np.array(preds))
test_y = torch.from_numpy(np.array(test_y))

In [ ]:
confusionMatrix = ConfusionMatrix(task = 'binary', threshold = 0.5)
confusionMatrix(preds, test_y)

tensor([[450, 104],
        [101, 447]])

In [ ]:
accuracy = Accuracy(task = 'binary', threshold = 0.5)
print("Accuracy:", accuracy(preds, test_y).item())

Accuracy: 0.8139746189117432


In [ ]:
precision = Precision(task = 'binary', threshold = 0.5)
recall = Recall(task = 'binary', threshold = 0.5)
f1Macro = F1Score(task = 'binary', threshold = 0.5)
print("Percision:", precision(preds, test_y).item(), "\nRecall:", recall(preds, test_y).item(), "\nF1 Score:", f1Macro(preds, test_y).item())

Percision: 0.8112522959709167 
Recall: 0.8156934380531311 
F1 Score: 0.8134667873382568


In [ ]:
auroc = AUROC(task = 'binary')
print("AUROC:", auroc(preds, test_y).item())

AUROC: 0.8139838576316833


# Complement Naive Bayes (ComplementNB)

**5 K-Fold Cross Validation (CV)**

In [ ]:
clf_ComplementNB = ComplementNB(alpha = 0.09115888299750818)

In [ ]:
val_acc, val_prec, val_rec, val_f1, val_auroc = runKFold(clf_ComplementNB, 'index', 5, False)

In [ ]:
val_acc

[0.8310679793357849,
 0.8132295608520508,
 0.7918288111686707,
 0.7879377603530884,
 0.7821011543273926]

In [ ]:
val_prec

[0.8167939186096191,
 0.8078431487083435,
 0.7865612506866455,
 0.7718631029129028,
 0.7713178396224976]

In [ ]:
val_rec

[0.8458498120307922,
 0.8142292499542236,
 0.7896825671195984,
 0.8055555820465088,
 0.7896825671195984]

In [ ]:
val_f1

[0.8310679793357849,
 0.8110235929489136,
 0.788118839263916,
 0.7883495092391968,
 0.7803921699523926]

In [ ]:
val_auroc

[0.8313218355178833,
 0.8132448196411133,
 0.79178786277771,
 0.7882739901542664,
 0.7822458744049072]

In [ ]:
acc_cv_mean = np.mean(val_acc)
prec_cv_mean = np.mean(val_prec)
rec_cv_mean = np.mean(val_rec)
f1_cv_mean = np.mean(val_f1)
auroc_cv_mean = np.mean(val_auroc)

print('5-Fold CV Mean')
print('5-Fold CV Accuracy: {}'.format(acc_cv_mean))
print('5-Fold CV Precision: {}'.format(prec_cv_mean))
print('5-Fold CV Recall: {}'.format(rec_cv_mean))
print('5-Fold CV F1 Score: {}'.format(f1_cv_mean))
print('5-Fold CV AUROC: {}'.format(auroc_cv_mean))

5-Fold CV Mean
5-Fold CV Accuracy: 0.8012330532073975
5-Fold CV Precision: 0.7908758521080017
5-Fold CV Recall: 0.8089999556541443
5-Fold CV F1 Score: 0.7997904181480407
5-Fold CV AUROC: 0.801374876499176


In [ ]:
acc_cv_std = np.std(val_acc)
prec_cv_std = np.std(val_prec)
rec_cv_std = np.std(val_rec)
f1_cv_std = np.std(val_f1)
auroc_cv_std = np.std(val_auroc)

print('5-Fold CV Standard Deviation')
print('5-Fold CV Accuracy: {}'.format(acc_cv_std))
print('5-Fold CV Precision: {}'.format(prec_cv_std))
print('5-Fold CV Recall: {}'.format(rec_cv_std))
print('5-Fold CV F1 Score: {}'.format(f1_cv_std))
print('5-Fold CV AUROC: {}'.format(auroc_cv_std))

5-Fold CV Standard Deviation
5-Fold CV Accuracy: 0.01824978437085815
5-Fold CV Precision: 0.018559372806591673
5-Fold CV Recall: 0.020704802066161418
5-Fold CV F1 Score: 0.018696963568070747
5-Fold CV AUROC: 0.018260222785614174


In [ ]:
cv_mean = ["cv mean", acc_cv_mean, prec_cv_mean, rec_cv_mean, f1_cv_mean, auroc_cv_mean]
cv_std = ["cv std", acc_cv_std, prec_cv_std, rec_cv_std, f1_cv_std, auroc_cv_std]

In [ ]:
five_k_cv_result_data = {'k': [1, 2, 3, 4, 5],
                          'val_acc': val_acc,
                          'val_prec': val_prec,
                          'val_rec': val_rec,
                          'val_f1': val_f1,
                          'val_auroc': val_auroc}

five_k_cv_result_df = pd.DataFrame(five_k_cv_result_data)

In [ ]:
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)

<ipython-input-94-f8ca0e644d46>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_mean)), ignore_index=True)
<ipython-input-94-f8ca0e644d46>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_k_cv_result_df = five_k_cv_result_df.append(dict(zip(five_k_cv_result_df.columns, cv_std)), ignore_index=True)


In [ ]:
five_k_cv_result_df

,k,val_acc,val_prec,val_rec,val_f1,val_auroc
0,1,0.831068,0.816794,0.845850,0.831068,0.831322
1,2,0.813230,0.807843,0.814229,0.811024,0.813245
2,3,0.791829,0.786561,0.789683,0.788119,0.791788
3,4,0.787938,0.771863,0.805556,0.788350,0.788274
4,5,0.782101,0.771318,0.789683,0.780392,0.782246
5,cv mean,0.801233,0.790876,0.809000,0.799790,0.801375
6,cv std,0.018250,0.018559,0.020705,0.018697,0.018260


**Final Build**

In [ ]:
train_X = fdt_hb_BABESG2_train[FEATURES]
test_X = fdt_hb_BABESG2_test[FEATURES]

train_y = fdt_hb_BABESG2_train['bias']
test_y = fdt_hb_BABESG2_test['bias']

In [ ]:
clf_ComplementNB = ComplementNB(alpha = 0.09115888299750818)

In [ ]:
trained_clf = clf_ComplementNB.fit(train_X, train_y)

In [ ]:
preds = trained_clf.predict(test_X)

In [ ]:
preds = torch.from_numpy(np.array(preds))
test_y = torch.from_numpy(np.array(test_y))

In [ ]:
confusionMatrix = ConfusionMatrix(task = 'binary', threshold = 0.5)
confusionMatrix(preds, test_y)

tensor([[445, 109],
        [105, 443]])

In [ ]:
accuracy = Accuracy(task = 'binary', threshold = 0.5)
print("Accuracy:", accuracy(preds, test_y).item())

Accuracy: 0.8058076500892639


In [ ]:
precision = Precision(task = 'binary', threshold = 0.5)
recall = Recall(task = 'binary', threshold = 0.5)
f1Macro = F1Score(task = 'binary', threshold = 0.5)
print("Percision:", precision(preds, test_y).item(), "\nRecall:", recall(preds, test_y).item(), "\nF1 Score:", f1Macro(preds, test_y).item())

Percision: 0.8025362491607666 
Recall: 0.8083941340446472 
F1 Score: 0.8054545521736145


In [ ]:
auroc = AUROC(task = 'binary')
print("AUROC:", auroc(preds, test_y).item())

AUROC: 0.8058216571807861
